In [1]:
import os
from io import BytesIO
import re
from urllib.parse import urlparse, urlunparse
import pandas as pd
import requests
import json
import time
from PIL import Image
from tqdm.auto import tqdm

In [2]:
proxies = {
    # 'http': 'http://localhost:7890',
    # 'https': 'http://localhost:7890'
    'http': 'http://10.68.98.153:7890',
    'https': 'http://10.68.98.153:7890'
}

headers = {'Content-Type': 'application/json'}

In [3]:
url = 'https://store.steampowered.com/api/appdetails/'

save_dir = '../dataset/scrap/steam/'

In [4]:
os.makedirs(os.path.join(save_dir, 'json'), exist_ok=True)
os.makedirs(os.path.join(save_dir, 'screenshot'), exist_ok=True)
os.makedirs(os.path.join(save_dir, 'header_image'), exist_ok=True)

In [5]:
js = json.load(open('../dataset/steam-games-dataset/games.json'))

In [6]:
def worker(appid):
    
    output_path = os.path.join(save_dir, 'json', f'{appid}.json')
    
    if os.path.exists(output_path):
        return True
    
    try:
        data = requests.get(f'{url}?appids={appid}', proxies=proxies)
        data = json.loads(data.content)
    except:
        return False
    
    json.dump(data, open(output_path, 'wt'))
    
    return True

In [7]:
file_names = list()
urls = list()
appids = list()

for appid, item in js.items():
    
    for url in item['screenshots']:
        file_names.append(os.path.split(urlparse(url).path)[-1])
        urls.append(url)
        appids.append(appid)

#     url = item['header_image']
    
#     file_names.append(os.path.split(urlparse(url).path)[-1])
#     urls.append(url)
#     appids.append(appid)

In [8]:
df = pd.DataFrame()

df['file_name'] = file_names
df['url'] = urls
df['appid'] = appids

df

,file_name,url,appid
0,0000005994.1920x1080.jpg,https://cdn.akamai.steamstatic.com/steam/apps/...,20200
1,0000005993.1920x1080.jpg,https://cdn.akamai.steamstatic.com/steam/apps/...,20200
2,0000005992.1920x1080.jpg,https://cdn.akamai.steamstatic.com/steam/apps/...,20200
3,0000006011.1920x1080.jpg,https://cdn.akamai.steamstatic.com/steam/apps/...,20200
4,0000005685.1920x1080.jpg,https://cdn.akamai.steamstatic.com/steam/apps/...,20200
...,...,...,...
622857,ss_95a3af03fa9d4325bd81c9274aa5fdef71f28aff.19...,https://cdn.akamai.steamstatic.com/steam/apps/...,2352890
622858,ss_e57da0f09a0cb30e7a572842d503741888837bf7.19...,https://cdn.akamai.steamstatic.com/steam/apps/...,2352890
622859,ss_78e7323b3bca5b179b5d27722ca3810384e5ed56.19...,https://cdn.akamai.steamstatic.com/steam/apps/...,2352890
622860,ss_fe177dba34a908a90dc911a9514594be12e7dde0.19...,https://cdn.akamai.steamstatic.com/steam/apps/...,2352890


In [9]:
def worker(url, output_path):
    
    if os.path.exists(output_path):
        return
    
    try:
        data = requests.get(url, proxies=proxies, timeout=(3.0, 10.0))
    except KeyboardInterrupt:
        return
    except:
        return

    if data.status_code != 200:
        return
    
    try:
        Image.open(BytesIO(data.content)).save(output_path)
    except:
        return
    
    return

In [ ]:
for url, file_name in tqdm(df[['url', 'file_name']].itertuples(index=False, name=None), total=df.shape[0]):
    output_path = os.path.join(save_dir, 'screenshot', file_name)
    worker(url, output_path)
    # time.sleep(0.5)

  0%|          | 0/622862 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

